In [1]:
import agate

In [2]:
population = agate.Table.from_csv('Downloads/population.csv')

In [3]:
print(population)

|------------------+---------------|
|  column_names    | column_types  |
|------------------+---------------|
|  STNAME          | Text          |
|  CTYNAME         | Text          |
|  POPESTIMATE2010 | Number        |
|  POPESTIMATE2011 | Number        |
|  POPESTIMATE2012 | Number        |
|  POPESTIMATE2013 | Number        |
|  POPESTIMATE2014 | Number        |
|------------------+---------------|



Create a variable for percent change. Within that variable, create a new column 'pechange1014' to include a percent change value in the new table. To get that percent change value, run an agate.PercentChange computation on the population table columns 'POPESTIMATE2010' and 'POPESTIMATE2014'. 

In [4]:
change1014 = population.compute([
        ('pechange1014', agate.PercentChange('POPESTIMATE2010', 'POPESTIMATE2014'))
    ])

Print the new table to check for errors.

In [6]:
change1014.print_table()

|-----------------------+-----------------------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------------------------|
|  STNAME               | CTYNAME                           | POPESTIMATE2010 | POPESTIMATE2011 | POPESTIMATE2012 | POPESTIMATE2013 | POPESTIMATE2014 |                      pechange1014  |
|-----------------------+-----------------------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------------------------|
|  Alabama              | Autauga County                    |          54,684 |          55,275 |          55,192 |          55,136 |          55,395 |      1.30019749835418038183015142  |
|  Alabama              | Baldwin County                    |         183,216 |         186,694 |         190,561 |         195,443 |         200,111 |      9.22135621343114138503187494  |
|  Alabama              | Barbour County               

Now it's time to narrow our focus to the relevant columns: in this case, 'STNAME', 'CTYNAME', and 'pechange1014'. To do this, create a for_printing variable by attaching a select function to the previous table, then list those three column names in the parentheses that follow.

In [7]:
for_printing = change1014.select(['STNAME', 'CTYNAME', 'pechange1014'])

Before we print our new, narrower table, let's sort it by percent change, starting with the highest value. Use an order_by function with the for_printing table to do this.

In [8]:
sorted_counties = for_printing.order_by('pechange1014', reverse=True)

Print the new sorted table.

In [9]:
sorted_counties.print_table()

|-----------------------+-----------------------------------+------------------------------------|
|  STNAME               | CTYNAME                           |                      pechange1014  |
|-----------------------+-----------------------------------+------------------------------------|
|  North Dakota         | McKenzie County                   |     71.94683346364347146207974980  |
|  North Dakota         | Williams County                   |     42.33819164488548265627076596  |
|  North Dakota         | Mountrail County                  |     26.62783171521035598705501618  |
|  North Dakota         | Stark County                      |     24.71564078347636841456904693  |
|  North Dakota         | Dunn County                       |     24.23044337757695566224230443  |
|  Florida              | Sumter County                     |     21.27864923054079565581681462  |
|  Louisiana            | St. Bernard Parish                |     20.68974888574845091857810632  |
|  Montana

WAY too many decimals. To round those percent change values to a more manageable number of sig digs, import the function Decimal from the python library decimal, then use that function to define a new rounded percent change function. Finally, create a new table using this round_pechange function.

In [10]:
from decimal import Decimal

def round_pechange(row):
    return row['pechange1014'].quantize(Decimal('0.1'))

rounded_change = sorted_counties.compute([
        ('round_pechange', agate.Formula(agate.Number(), round_pechange))
    ])



Just as we did before, we need to limit our table domain to the relevant columns (STNAME, CTYNAME, round_pechange). Then print.

In [15]:
for_round_printing = rounded_change.select(['STNAME', 'CTYNAME', 'round_pechange'])

In [16]:
for_round_printing.print_table()

|-----------------------+-----------------------------------+-----------------|
|  STNAME               | CTYNAME                           | round_pechange  |
|-----------------------+-----------------------------------+-----------------|
|  North Dakota         | McKenzie County                   |           71.9  |
|  North Dakota         | Williams County                   |           42.3  |
|  North Dakota         | Mountrail County                  |           26.6  |
|  North Dakota         | Stark County                      |           24.7  |
|  North Dakota         | Dunn County                       |           24.2  |
|  Florida              | Sumter County                     |           21.3  |
|  Louisiana            | St. Bernard Parish                |           20.7  |
|  Montana              | Richland County                   |           18.9  |
|  Texas                | Andrews County                    |           17.8  |
|  Texas                | Sterling Count